In [15]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset

class RetinaDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        left_img_path = os.path.join(self.root_dir, 'left', self.annotations.iloc[index, 0] + '.jpg')  # Assuming the extension is .jpeg
        right_img_path = os.path.join(self.root_dir, 'right', self.annotations.iloc[index, 1] + '.jpg')  # Assuming the extension is .jpeg
        
        left_image = Image.open(left_img_path).convert('RGB')
        right_image = Image.open(right_img_path).convert('RGB')
        
        if self.transform:
            left_image = self.transform(left_image)
            right_image = self.transform(right_image)

        # You can decide how to return these images, here I'm returning them as a tuple
        return (left_image, right_image)
    
# Contrastive Loss
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive


In [16]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = RetinaDataset(csv_file='train.csv', root_dir='train', transform=transform)


In [17]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


In [18]:
import torch.nn as nn
import torchvision.models as models

# 加载预训练的ResNet-18模型
model = models.resnet18(pretrained=True)

# 替换最后的全连接层
num_features = model.fc.in_features
num_classes = 2  # 假设我们有两个类别：动脉和静脉
model.fc = nn.Linear(num_features, num_classes)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [20]:
from torch.utils.data import DataLoader, random_split

# 将数据集分为训练集和验证集
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)


In [22]:
def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Validation Accuracy: {100 * correct / total:.2f}%')


In [23]:
train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10)


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [10]:
# 示例代码，检查train_loader的输出
images, labels = next(iter(train_loader))
print(images.shape)  # 应该是 [batch_size, channels, height, width]
print(labels.shape)  # 应该是 [batch_size]


torch.Size([32, 3, 224, 224])
torch.Size([32, 3, 224, 224])


In [12]:
images, labels = next(iter(train_loader))
print(labels.shape)



torch.Size([32, 3, 224, 224])


In [ ]:
class RetinaTestDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.file_names = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.file_names.iloc[index, 0])
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = RetinaTestDataset(csv_file='test_candidates.csv', root_dir='test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
def predict_and_save(model, test_loader, output_file='predictions.csv'):
    model.eval()
    predictions = []

    with torch.no_grad():
        for images in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.cpu().numpy())

    # 保存预测结果为CSV文件
    df = pd.DataFrame({
        'filename': test_dataset.file_names.iloc[:, 0],
        'label': predictions
    })

    df.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")

predict_and_save(model, test_loader)


In [1]:
import os

# 获取当前工作目录
current_working_directory = os.getcwd()
print(current_working_directory)
# 获取当前工作目录中的文件和文件夹
files_and_directories = os.listdir(current_working_directory)

# 打印当前工作目录中的文件和文件夹
for item in files_and_directories:
    print(item)


.DS_Store
new3.ipynb
test
Untitled.ipynb
sample-submission.csv
testkernel.ipynb
train
try.ipynb
train.csv
.ipynb_checkpoints
test_candidates.csv
resnet18_weights.pth
.git
data
